# KOBERT 사용
- 출처 : 
  - https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf
  - https://www.dinolabs.ai/m/271

## 1. 환경 준비

In [12]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install kobert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [13]:
!pip install 'git+https://git@github.com/SKTBrain/KoBERT.git@master'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-_7iah7xg
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-_7iah7xg


In [14]:
pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-8s27zmg2/kobert-tokenizer_6befc3d37b784496bd645c707864752f
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-8s27zmg2/kobert-tokenizer_6befc3d37b784496bd645c707864752f


In [15]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [16]:
pip install urllib3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from kobert.utils import get_tokenizer

In [18]:
from kobert_tokenizer import KoBERTTokenizer

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [19]:
import torch
from kobert import get_pytorch_kobert_model

In [20]:
#GPU 사용
device = torch.device("cuda:0")

In [21]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


## 2. 데이터셋 전처리

In [22]:
import pandas as pd
chatbot_data = pd.read_csv('/content/train_data.csv')

In [23]:
len(chatbot_data)

81758

In [24]:
chatbot_data.sample(n=10)

,Unnamed: 0,감정_대분류,감정_소분류,사람문장1,label
71454,71454,불안,두려운,젊은 사람들 사이에서 설 자리가 없네.,0
32915,32915,분노,좌절한,나이가 들다 보니 건강이 안 좋아서 너무 좌절해 버렸어.,0
61735,61735,기쁨,만족스러운,친구들은 월급으로 전전긍긍하면서 사는데 나는 부모님이 덕택에 의식주 걱정 없이 사니...,1
73262,73262,상처,배신당한,나도 같이 준비했는데 누나 거짓말 때문에 아빠가 누나에게만 고마워하는 것 같아서 속상해.,0
68879,68879,슬픔,슬픔,나에게도 그런 일이 생길까 봐 두렵고 초조해.,0
33907,33907,분노,좌절한,월급쟁이 살림 평생 아껴 쓰고 살았는데 이제 은퇴하면 어떻게 살아.,0
17829,17829,기쁨,기쁨,아들에게 생일 선물로 냉장고를 받아서 정말 행복해!,1
8036,8036,기쁨,자신하는,얼마 전 동창회에 다녀왔는데 나와 아내만 삼십 대처럼 보여서 정말 기분이 좋았어.,1
60113,60113,기쁨,신뢰하는,팀장님이 모두가 다 반대하는 일을 하겠다고 하시는 데는 이유가 있을 것 같았거든.,1
4972,4972,불안,두려운,건강 때문에 회사를 빠져야 한다고 보고하기 무서워.,0


In [25]:
chatbot_data.drop('Unnamed: 0', axis=1,inplace=True)

In [26]:
chatbot_data[:5]

,감정_대분류,감정_소분류,사람문장1,label
0,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,1
1,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,0
2,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,0
3,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,1
4,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,1


In [27]:
chatbot_data['감정_대분류'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처', '불안 ', '기쁨 '], dtype=object)

In [28]:
chatbot_data.loc[(chatbot_data['감정_대분류'] == "기쁨"), '감정_대분류'] = 0  #기쁨 => 0
chatbot_data.loc[(chatbot_data['감정_대분류'] == '기쁨 '), '감정_대분류'] = 0  #기쁨 => 0
chatbot_data.loc[(chatbot_data['감정_대분류'] == "불안"), '감정_대분류'] = 1  #불안 => 1
chatbot_data.loc[(chatbot_data['감정_대분류'] == "불안 "), '감정_대분류'] = 1  #불안 => 1
chatbot_data.loc[(chatbot_data['감정_대분류'] == "당황"), '감정_대분류'] = 2  #당황 => 2
chatbot_data.loc[(chatbot_data['감정_대분류'] == "슬픔"), '감정_대분류'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['감정_대분류'] == "분노"), '감정_대분류'] = 4  #분노 => 4
chatbot_data.loc[(chatbot_data['감정_대분류'] == "상처"), '감정_대분류'] = 5  #상처 => 5

In [29]:
chatbot_data['감정_대분류'].unique()

array([0, 1, 2, 3, 4, 5], dtype=object)

In [30]:
chatbot_data.loc[(chatbot_data['감정_대분류'] == "당황"), '감정_대분류'] = 2  #당황 => 2

In [31]:
chatbot_data['감정_대분류'].unique()

array([0, 1, 2, 3, 4, 5], dtype=object)

In [32]:
data_list = []
for q, label in zip(chatbot_data['사람문장1'], chatbot_data['감정_대분류'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [33]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['아내가 드디어 출산하게 되어서 정말 신이 나.', '0']
['주변 사람들이 나에게 무능하다고 말해서 씁쓸해.', '3']
['엄마가 지금이라도 공부 말고 다른 길을 알아보자고 하셔서 슬퍼.', '3']
['오늘 면접을 보고 왔는데 대답을 잘 못 해서 나 자신한테 실망스러워.', '3']
['요즘 영어 수학 점수도 떨어지고 학원에 가면 스트레스만 받는 것 같아.', '0']
['아니 사람들은 왜 자기 일도 아니면서 참견하고 난리야?', '4']
['카페에 가서 대화하거나 같이 술 마시면서 이야기했던 것 같아.', '1']


## 3. Train data & Test Data

In [34]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [35]:
print(len(dataset_train))
print(len(dataset_test))

61318
20440


## 4. KoBERT 입력 데이터로 만들기

In [36]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
 

In [37]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [38]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [39]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [40]:
data_train[0]

(array([   2, 3647, 1397, 5330,  517, 5890, 1493, 7289, 6116, 4136, 4998,
        5737, 7318, 2086, 5007, 5176, 7953, 5876,  517,   54,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(21, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [41]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## 5. KoBERT 학습모델 만들기

In [42]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [43]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)


In [44]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [45]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [46]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [47]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [48]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [49]:
train_dataloader

## 6. KoBERT 모델 학습시키기

In [50]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/959 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.9489400386810303 train acc 0.15625
epoch 1 batch id 201 loss 1.5635441541671753 train acc 0.2543532338308458
epoch 1 batch id 401 loss 1.2598744630813599 train acc 0.37800031172069826
epoch 1 batch id 601 loss 1.091031789779663 train acc 0.4293365224625624
epoch 1 batch id 801 loss 1.383296251296997 train acc 0.457748127340824
epoch 1 train acc 0.471584984358707


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.5837076822916667


  0%|          | 0/959 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.887535572052002 train acc 0.703125
epoch 5 batch id 201 loss 0.7583152055740356 train acc 0.7400497512437811
epoch 5 batch id 401 loss 0.5154836177825928 train acc 0.7472334788029925
epoch 5 batch id 601 loss 0.5246155261993408 train acc 0.7540817387687188
epoch 5 batch id 801 loss 0.7215061783790588 train acc 0.7568469101123596
epoch 5 train acc 0.7564520333680917


  0%|          | 0/320 [00:00<?, ?it/s]

epoch 5 test acc 0.584521484375


In [51]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
pip install model1

In [53]:
import model1

In [77]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            logits = np.argmax(logits)
            

            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("불안이")
            elif np.argmax(logits) == 2:
                test_eval.append("당황이")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("분노가")
            elif np.argmax(logits) == 5:
                test_eval.append("상처가")
            

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

In [79]:
import logging

from transformers.configuration_utils import PretrainedConfig
from transformers import BertModel, BertConfig, GPT2Config



logger = logging.getLogger(__name__)

#KoBERT
kobert_config = {
    'attention_probs_dropout_prob': 0.1,
    'hidden_act': 'gelu',
    'hidden_dropout_prob': 0.1,
    'hidden_size': 768,
    'initializer_range': 0.02,
    'intermediate_size': 3072,
    'max_position_embeddings': 512,
    'num_attention_heads': 12,
    'num_hidden_layers': 12,
    'type_vocab_size': 2,
    'vocab_size': 8002
}
#KoGPT2
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000,
    "activation_function": "gelu"
}

def get_kobert_config():
    return BertConfig.from_dict(kobert_config)

def get_kogpt2_config():
    return GPT2Config.from_dict(kogpt2_config)

In [80]:
import torch
import torch.nn as nn
import random
import model1

# from model.kobert import KoBERTforSequenceClassfication, kobert_input
from kobert_transformers import get_tokenizer

def load_answer():
  root_path = '..'
  answer_path = f"{root_path}/data/chatbot_wellness_category.txt"

  a_f = open(answer_path,'r')
  answer_lines = a_f.readlines()
  answer = {}

  for line_num, line_data in enumerate(answer_lines):
    data = line_data.split('    ')

    answer[data[1][:-1]] =data[0]

  return  answer

if __name__ == "__main__":
  root_path='..'
  checkpoint_path =f"{root_path}/checkpoint"
  save_ckpt_path = f"{checkpoint_path}/kobert-chatbot-wellness.pth"

  #답변 불러오기
  answer = load_answer()

  ctx = "cuda" if torch.cuda.is_available() else "cpu"
  device = torch.device(ctx)

  # 저장한 Checkpoint 불러오기
  checkpoint = torch.load(save_ckpt_path, map_location=device)

  model = BERTClassifier(num_labels=9322)
  model.load_state_dict(checkpoint['model_state_dict'])

  model.eval()

  tokenizer = get_tokenizer()

  while 1:
    sent = input('\nQuestion: ') # '요즘 기분이 우울한 느낌이에요'
    data = kobert_input(tokenizer,sent, device,512)
    # print(data)

    output = model(**data)

    logit = output
    softmax_logit = nn.Softmax(logit).dim
    softmax_logit = softmax_logit[0].squeeze()

    max_index = torch.argmax(softmax_logit).item()
    max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()

    print(f'Answer: {answer[str(max_index)]}, index: {max_index}, value: {max_index_value}')
    print('-'*50)
  # print('argmin:',softmax_logit[torch.argmin(softmax_logit)])





FileNotFoundError: ignored